In [1]:
import pandas as pd
import glob

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
SIREN = pd.read_csv(r'C:\Users\RémiJAYLET\Documents\City&You\0_data\base_sirene\StockUniteLegale_utf8.csv', 
                    usecols=['siren', 'denominationUniteLegale'],
                    dtype = {'siren': str, 'denominationUniteLegale': str})

In [26]:
SIREN = SIREN[~SIREN.denominationUniteLegale.isna()]

In [27]:
len(SIREN)

13010157

In [ ]:
df_parc1 = pd.concat([pd.read_csv(i, sep=';', encoding='latin', low_memory=False) for i in glob.glob(
    "../0_data/parcelles/Fichier des parcelles (situation 2022)-dept 01 à 61/Fichier des parcelles (situation 2022)-dpts 01 à 61/*.txt",
    recursive=True)])
df_parc2 = pd.concat([pd.read_csv(i, sep=';', encoding='latin', low_memory=False) for i in glob.glob(
    "../0_data/parcelles/Fichier des parcelles (situation 2022)-dept 62 à 976/Fichier des parcelles (situation 2022)-dpts 62 à 976/*.txt",
    recursive=True)])
df_parc = pd.concat([df_parc1, df_parc2], axis=0)

In [30]:
# get rid of unknown SIREN
parcelles = df_parc.dropna(subset='N° SIREN (Propriétaire(s) parcelle)')
parcelles = parcelles[['N° SIREN (Propriétaire(s) parcelle)', 'Dénomination (Propriétaire(s) parcelle)', 'Nom Commune (Champ géographique)']]
parcelles = parcelles.drop_duplicates()

In [31]:
len(parcelles)

2484385

### Match with SIREN CODE

In [32]:
df_merge_siren = parcelles.merge(SIREN,
                             how='inner',
                             left_on='N° SIREN (Propriétaire(s) parcelle)',
                             right_on='siren')
df_merge_siren = df_merge_siren.drop_duplicates()[['siren', 'Dénomination (Propriétaire(s) parcelle)', 'denominationUniteLegale']]

In [33]:
df_merge_siren

,siren,Dénomination (Propriétaire(s) parcelle),denominationUniteLegale
0,443843081,DE LORDRE,GROUPEMENT FORESTIER DE L'ORDRE
1,443843081,DE LORDRE,GROUPEMENT FORESTIER DE L'ORDRE
2,343954038,DE MONT JOLI,GFA DE MONT JOLI
3,343954038,DE MONT JOLI,GFA DE MONT JOLI
4,343954038,DE MONT JOLI,GFA DE MONT JOLI
...,...,...,...
1698450,888910486,SCI MUMANGA,SCI MUMANGA
1698451,888910445,SCI KIWANO,SCI KIWANO
1698452,888910478,SCI MARASHI,SCI MARASHI
1698453,847712734,HANAA,HANAA


### Match with name

In [34]:
# Number of missing SIREN in the PARCELLES table
len(parcelles[parcelles['N° SIREN (Propriétaire(s) parcelle)'].str.startswith('U', na=False)])

783447

In [35]:
#names of the companies to find
nom_entreprise = list(parcelles.loc[(parcelles['N° SIREN (Propriétaire(s) parcelle)'].str.startswith('U', na=False))
                                   & ~(parcelles['Dénomination (Propriétaire(s) parcelle)'].str.startswith('PROPRIETAIRES ', na=False))]['Dénomination (Propriétaire(s) parcelle)'].unique())

In [39]:
df_merge_nom = parcelles.merge(SIREN[:10000], how='left',
                           left_on=['Dénomination (Propriétaire(s) parcelle)'],
                           right_on=['denominationUniteLegale'])
df_merge_nom = df_merge_nom[~df_merge_nom.siren.isna()]

In [41]:
df_merge_nom

,N° SIREN (Propriétaire(s) parcelle),Dénomination (Propriétaire(s) parcelle),Nom Commune (Champ géographique),siren,denominationUniteLegale
1402,016250029,APRR,APREMONT,016250029,APRR
1502,016250029,APRR,ARBENT,016250029,APRR
2623,016250029,APRR,BAGE-DOMMARTIN,016250029,APRR
2783,016250029,APRR,BALAN,016250029,APRR
2874,016250029,APRR,BEAUPONT,016250029,APRR
...,...,...,...,...,...
2483860,U23720813,SYNDICAT DES COPROPRIETAIRES,SAINTE SUZANNE,010242147,SYNDICAT DES COPROPRIETAIRES
2483861,U23720813,SYNDICAT DES COPROPRIETAIRES,SAINTE SUZANNE,010248359,SYNDICAT DES COPROPRIETAIRES
2483862,U23720813,SYNDICAT DES COPROPRIETAIRES,SAINTE SUZANNE,010248789,SYNDICAT DES COPROPRIETAIRES
2483863,U23720813,SYNDICAT DES COPROPRIETAIRES,SAINTE SUZANNE,010248797,SYNDICAT DES COPROPRIETAIRES


In [53]:
df_merge_known = df_merge_nom[~df_merge_nom['N° SIREN (Propriétaire(s) parcelle)'].str.startswith('U', na=False)]
print('# matchs :',  len(df_merge_known[df_merge_known['N° SIREN (Propriétaire(s) parcelle)'] == df_merge_known['siren']]))
print('# errors :',  len(df_merge_known[df_merge_known['N° SIREN (Propriétaire(s) parcelle)'] != df_merge_known['siren']]))
print('success rate :', round(100 * len(df_merge_known[df_merge_known['N° SIREN (Propriétaire(s) parcelle)'] == df_merge_known['siren']])/len(df_merge_known),2), '%')

# matchs : 1189
# errors : 2815
success rate : 29.7 %


In [55]:
df_merge_unknown = df_merge_nom[df_merge_nom['N° SIREN (Propriétaire(s) parcelle)'].str.startswith('U', na=False)]
print('# unknown siren "retrouvé" :', len(df_merge_unknown))
print('taux match :', len(df_merge_unknown) / len() )

# unknown siren "retrouvé" : 1901


In [ ]:
df_merge_nom.loc[(df_merge_nom.siren.notna()) & (df_merge_nom['N° SIREN (Propriétaire(s) parcelle)'].str.startswith('U', na=False))].head(10)

In [ ]:
len(df_merge_nom.loc[(df_merge_nom.siren.notna()) & (df_merge_nom['N° SIREN (Propriétaire(s) parcelle)'].str.startswith('U', na=False))])

In [ ]:
tmp = list(SIRENE['denominationUniteLegale'].copy())
SIRENE['denominationUniteLegale_formatted'] = [str(ele).replace('SC ', '').replace('SCI ', '').replace('SA ', '').replace('SCP ', '').replace('SARL ', '').replace('GFA ', '').replace('SCA ', '').replace('.', ' ').replace("'", ' ').replace('-', ' ') for ele in tmp]

tmp = list(parcelles['Dénomination (Propriétaire(s) parcelle)'].copy())
parcelles['Dénomination (Propriétaire(s) parcelle)_formatted'] = [str(ele).replace('SC ', '').replace('SCI ', '').replace('SA ', '').replace('SCP ', '').replace('SARL ', '').replace('GFA ', '').replace('SCA ', '').replace('.', ' ').replace("'", ' ').replace('-', ' ') for ele in tmp]

In [ ]:
print('prop modif parcelles : ', round(100* len(parcelles.loc[~(parcelles['Dénomination (Propriétaire(s) parcelle)'] == parcelles['Dénomination (Propriétaire(s) parcelle)_formatted'])]) / len(parcelles), 2), '%')
print('prop modif SIRENE : ', round(100* len(SIRENE.loc[~(SIRENE['denominationUniteLegale'] == SIRENE['denominationUniteLegale_formatted'])]) / len(SIRENE), 2), '%')

In [ ]:
df_merge = parcelles.merge(SIRENE, how='left',
                           left_on=['Dénomination (Propriétaire(s) parcelle)_formatted', 'Nom voie (Adresse parcelle)', 'Nom Commune (Champ géographique)'],
                           right_on=['denominationUniteLegale_formatted', 'libelleVoieEtablissement', 'libelleCommuneEtablissement'])

In [ ]:
df_merge_siren = parcelles.merge(SIRENE, how='left',
                     left_on='N° SIREN (Propriétaire(s) parcelle)',
                     right_on='siren')

MemoryError: Unable to allocate 8.91 GiB for an array with shape (1196362999,) and data type int64

In [ ]:
df_merge_siren[df_merge_siren.siren.notna()][['Dénomination (Propriétaire(s) parcelle)_formatted', 'denominationUniteLegale_formatted']].head(50)

In [ ]:
len(df_merge.loc[(df_merge.siren.notna()) & (df_merge['N° SIREN (Propriétaire(s) parcelle)'].str.startswith('U', na=False))])

16036

Pistes pour retrouver plus d'adresses : 
- enlever la mention du type d'entreprise dand la denomination de la base SIREN (CSI, ...)
- contains plutot que egal